In [26]:
# Import the libraries
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
from datetime import datetime
import pytz
import sqlalchemy

# Define a dataframe with cities, latitude and longitude
data_df = pd.DataFrame()
city = ['Berlin', 'Paris', 'Madrid']
latitude = [52.52000, 48.85667, 40.41694]
longitude = [13.40500, 2.35222, -3.70333]
    
data_df['city'] = city
data_df['latitude'] = latitude
data_df['longitude'] = longitude

# Function to get the airpots with the database
def get_airports_loop(data_df):
    # Create a blank dictionary with the information to be requested
    airports_dict = {
        'city': [],
        'iataCode': [],
        'icaoCode': [],
        'name': [],
        'alpha2countryCode': [],
        'latitude': [],
        'longitude': []
    }
    # Declare the urls and the API key
    url = "https://aviation-reference-data.p.rapidapi.com/airports/search"
    headers = {
        "X-RapidAPI-Key": "API Key",
        "X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
    }
    # Create a json with the responses
    for i in range(len(data_df)):
        querystring = {"lat": data_df['latitude'][i], "lon": data_df['longitude'][i], "radius": "100"}
        response = requests.get(url, headers=headers, params=querystring)
        airports_json = response.json()
        
        # Create a loop to get the necesary information and store them into the dictionary
        for j in airports_json:
            airports_dict['city'].append(data_df['city'][i])
            airports_dict['iataCode'].append(j.get('iataCode', ''))
            airports_dict['icaoCode'].append(j.get('icaoCode', ''))
            airports_dict['name'].append(j.get('name', ''))
            airports_dict['alpha2countryCode'].append(j.get('alpha2countryCode', ''))
            airports_dict['latitude'].append(j.get('latitude', ''))
            airports_dict['longitude'].append(j.get('longitude', ''))

    return pd.DataFrame(airports_dict)

cities_airports = get_airports_loop(data_df)

# Changing the names of the cities for an id
data_df.index = data_df.index + 1
cities_airports['city_id'] = cities_airports['city_id'].map(data_df.reset_index().set_index('city')['index'])
# Cleaning the data
cities_airports.drop(['iataCode', 'alpha2countryCode', 'latitude', 'longitude'], axis=1, inplace=True)
cities_airports.rename(columns={'city': 'city_id'}, inplace=True)
cities_airports.rename(columns={'icaoCode': 'airport_icao', 'name': 'airport_name'}, inplace=True)
cities_airports = cities_airports.dropna(subset= 'airport_icao')

# Copying the dataframe to MySQL
schema="mydb" 
host="127.0.0.1"
user="root"
password="*******"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

cities_airports.to_sql('weather', 
                if_exists='append',
                con=con,
                index=False)

The response status is: 200
The response status is: 200
The response status is: 200


,city_id,airport_icao,airport_name
1,1,EDDB,SCHOENEFELD
3,1,EDDT,TEGEL
5,1,EDAY,STRAUSBERG
6,1,EDAX,LAERZ
7,2,LFOB,Paris Beauvais-Tille Airport
8,2,LFPG,Paris/ Ch.de Gaulle
9,2,LFPN,Toussus-Le-Noble
10,2,LFPC,Creil
12,2,LFPV,Paris Villacoublay-Velizy Villacoublay-
13,2,LFOE,Evreux Fauville
